In [1]:
# Importo librerias
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [2]:
# Ruta de la base de datos
root_dir = "C:/Users/BiDAlab/Proy_ASMI/e-BioDigit_DB/e-BioDigit_DB/"

In [3]:
# Selecciono un archivo de prueba

file_path = os.path.join(root_dir, "u101_digit_0_002.txt")

In [4]:
# Directorio de salida para imágenes
output_folder = "Converted_images"
os.makedirs(output_folder, exist_ok=True)

# Función para convertir coordenadas a una imagen 64x64
def coordenadas_a_imagen(coords, img_size=64, line_thickness=2, discontinuity_threshold=10):
    """
    Convierte coordenadas X, Y en una imagen binaria, conectando puntos consecutivos con líneas.
    
    Args:
        coords: Array de coordenadas (x, y).
        img_size: Tamaño de la imagen (28x28 por defecto).
        line_thickness: Grosor de las líneas.
        discontinuity_threshold: Umbral para detectar discontinuidades (levantar el lápiz).
    
    Returns:
        Imagen binaria (numpy array).
    """
    # Crear una imagen en blanco (fondo blanco)
    img = np.ones((img_size, img_size), dtype=np.uint8) * 255  # Blanco

    # Si no hay coordenadas, devolver imagen vacía
    if len(coords) == 0:
        return img

    # Normalizar coordenadas a [0, img_size-1]
    x_min, x_max = np.min(coords[:, 0]), np.max(coords[:, 0])
    y_min, y_max = np.min(coords[:, 1]), np.max(coords[:, 1])

    # Evitar división por cero
    if x_max == x_min:
        x_max = x_min + 1
    if y_max == y_min:
        y_max = y_min + 1

    # Escalar las coordenadas
    coords_scaled = np.zeros_like(coords)
    coords_scaled[:, 0] = (coords[:, 0] - x_min) / (x_max - x_min) * (img_size - 1)
    coords_scaled[:, 1] = (coords[:, 1] - y_min) / (y_max - y_min) * (img_size - 1)

    # Convertir coordenadas a enteros
    coords_scaled = coords_scaled.astype(int)

    # Dibujar líneas entre puntos consecutivos
    for i in range(len(coords_scaled) - 1):
        x1, y1 = coords_scaled[i]
        x2, y2 = coords_scaled[i + 1]

        # Calcular la distancia entre puntos consecutivos
        distance = np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

        # Si la distancia es menor que el umbral, dibujar una línea
        if distance < discontinuity_threshold:
            cv2.line(img, (x1, y1), (x2, y2), color=0, thickness=line_thickness)

    return img

# Recorrer todos los usuarios (ej. 101, 102, ..., 208)
for user in os.listdir(root_dir):
    user_path = os.path.join(root_dir, user)
    if not os.path.isdir(user_path):  # Saltar si no es un directorio
        continue
    
    # Recorrer las sesiones (session_1 y session_2)
    for session in ["session_1", "session_2"]:
        session_path = os.path.join(user_path, session)
        if not os.path.exists(session_path):  # Saltar si la sesión no existe
            continue

        # Procesar todos los archivos .txt dentro de la sesión
        for file_name in os.listdir(session_path):
            if file_name.endswith(".txt"):
                file_path = os.path.join(session_path, file_name)
                
                # Extraer información del nombre del archivo
                parts = file_name.split("_")
                digit = parts[2]  # Número (0-9)
                sample = parts[3].split(".")[0]  # Número de muestra
                
                # Leer archivo y extraer coordenadas
                with open(file_path, "r") as f:
                    lines = f.readlines()
                
                num_points = int(lines[0].strip())  # Número de puntos
                coords = np.array([list(map(float, line.split()[:2])) for line in lines[1:num_points+1]])

                # Convertir coordenadas a imagen
                img = coordenadas_a_imagen(coords, img_size=28, line_thickness=2, discontinuity_threshold=10)

                # Guardar la imagen con el mismo formato de nombres
                output_path = os.path.join(output_folder, f"{digit}")
                os.makedirs(output_path, exist_ok=True)
                output_path = os.path.join(output_path, f"{user}_{session}_{digit}_{sample}.png")
                # Crea la carpeta si no existe
                
                cv2.imwrite(output_path, img)

print("✅ Todas las imágenes han sido generadas correctamente.")

✅ Todas las imágenes han sido generadas correctamente.


In [5]:
print(output_path)

Converted_images\9\208_session_2_9_016.png
